# Cargar datos #

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pathlib import Path

def preparar_datos_para_clustering(df, columnas_excluir=None):
    """
    Prepara un DataFrame para clustering jerárquico:
    - Filtra numéricas
    - Divide por el número de jornadas según la liga
    - Aplica StandardScaler

    Parámetros:
    - df: DataFrame original con columna 'Liga'
    - columnas_excluir: lista de columnas a mantener fuera (ID, etiquetas)

    Retorna:
    - df_escalado: array numpy listo para clustering
    - df_escalado_df: DataFrame escalado con índice original
    - df_normalizado: DataFrame normalizado por jornadas
    """
    if columnas_excluir is None:
        columnas_excluir = []

    # Diccionario de jornadas por liga
    jornadas_por_liga = {
        'Bundesliga': 34,
        'Femenino': 30,
        'La Liga': 38,
        'Ligue 1': 34,  # Nota: Ligue 1 tiene 34 jornadas, no 38
        'Premier League': 38,
        'Serie A': 38
    }

    # Separar columnas excluidas
    df_excluir = df[columnas_excluir] if columnas_excluir else pd.DataFrame(index=df.index)

    # Seleccionar solo numéricas no excluidas
    df_numeric = df.select_dtypes(include=[np.number]).drop(columns=columnas_excluir, errors='ignore')

    # Normalizar por el número de jornadas según la liga
    df_normalizado = df_numeric.astype(float).copy()
    
    for liga, jornadas in jornadas_por_liga.items():
        # Encontrar índices de equipos de esta liga
        indices_liga = df[df['liga'] == liga].index
        # Dividir las estadísticas por el número de jornadas
        df_normalizado.loc[indices_liga] = df_numeric.loc[indices_liga] / jornadas

    # Escalar
    scaler = StandardScaler()
    df_escalado = scaler.fit_transform(df_normalizado)

    # Convertimos a DataFrame para conservar estructura
    df_escalado_df = pd.DataFrame(df_escalado, index=df.index, columns=df_normalizado.columns)

    return df_escalado, df_escalado_df, df_normalizado

In [3]:
BASE_DIR = Path.cwd().parent  # Asume que el notebook está en src/ y la raíz es la carpeta padre
INPUT_FILE = BASE_DIR / "output" / "Correlation" / "filtered_features.csv"

df = pd.read_csv(INPUT_FILE)
# Preprocesar (excluyendo columnas no numéricas o IDs)
X, df_ready, X_90 = preparar_datos_para_clustering(df, columnas_excluir=["Squad", "season","liga"])

In [4]:
# Configuración de ligas y número de clusters óptimo
ligas_config = {
    'Femenino': {
        'Por 90 min': {'clusters': 3},
        'Normalizado': {'clusters': 4}
    },
    'La Liga': {
        'Por 90 min': {'clusters': 3},
        'Normalizado': {'clusters': 4}
    },
    'Bundesliga': {
        'Por 90 min': {'clusters': 4},
        'Normalizado': {'clusters': 3}
    },
    'Ligue 1': {
        'Por 90 min': {'clusters': 3},
        'Normalizado': {'clusters': 2}
    },
    'Serie A': {
        'Por 90 min': {'clusters': 5},
        'Normalizado': {'clusters': 3}
    },
    'Premier League': {
        'Por 90 min': {'clusters': 3},
        'Normalizado': {'clusters': 3}
    },
    'Global': {
        'Por 90 min': {'clusters': 3},
        'Normalizado': {'clusters': 3}
    }
}


## Aplicar K-Means
Aplicamos K-means a los dos datasets, primero al normalizado y segundo al que almacena los datos por partido

In [52]:
from sklearn.cluster import KMeans

# Aseguramos que los índices coincidan
assert (df.index == df_ready.index).all()
assert (df.index == X_90.index).all()

# Creamos copias de los DataFrames para almacenar clusters
df_ready_clusters = df_ready.copy()
X_90_clusters = X_90.copy()

for liga, config in ligas_config.items():
    for tipo, valores in config.items():
        n_clusters = valores["clusters"]

        print(f"Procesando {liga} ({tipo}) con {n_clusters} clusters...")

        # Filtrar datos de la liga o usar todos si es Global
        if liga == "Global":
            if tipo == "Normalizado":
                X_liga = df_ready.copy()
            else:  # Por 90 min
                X_liga = X_90.copy()
            mask = df.index
        else:
            mask = df["liga"] == liga
            if tipo == "Normalizado":
                X_liga = df_ready.loc[mask]
            else:
                X_liga = X_90.loc[mask]

        # Entrenar KMeans
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(X_liga)

        # Crear nombre de columna
        col_name = f"Cluster_{tipo.replace(' ', '')}_{liga.replace(' ', '_')}"

        # Guardar resultados en los tres DataFrames
        df.loc[mask, col_name] = clusters + 1
        if tipo == "Normalizado":
            df_ready_clusters.loc[mask, col_name] = clusters + 1
        else:
            X_90_clusters.loc[mask, col_name] = clusters + 1

# Añadir columnas de contexto a df_ready_clusters y X_90_clusters
for df_cluster in [df_ready_clusters, X_90_clusters]:
    df_cluster["Squad"] = df["Squad"]
    df_cluster["season"] = df["season"]
    df_cluster["liga"] = df["liga"]

# Ahora sí podemos exportar
cols_clave = ["Squad", "season", "liga"]

df_ready_export = df_ready_clusters[cols_clave + [c for c in df_ready_clusters.columns if c.startswith("Cluster_")]]
X_90_export = X_90_clusters[cols_clave + [c for c in X_90_clusters.columns if c.startswith("Cluster_")]]
df_export = df[cols_clave + [c for c in df.columns if c.startswith("Cluster_")]]

# Guardar CSVs
df_export.to_csv("resultados_kmeans_clusters.csv", index=False)
df_ready_export.to_csv("resultados_kmeans_clusters_normalizado.csv", index=False)
X_90_export.to_csv("resultados_kmeans_clusters_X90.csv", index=False)

print("✅ Exportación completada. Todos los CSVs listos para Power BI.")



Procesando Femenino (Por 90 min) con 3 clusters...
Procesando Femenino (Normalizado) con 4 clusters...
Procesando La Liga (Por 90 min) con 3 clusters...
Procesando La Liga (Normalizado) con 4 clusters...
Procesando Bundesliga (Por 90 min) con 4 clusters...
Procesando Bundesliga (Normalizado) con 3 clusters...
Procesando Ligue 1 (Por 90 min) con 3 clusters...
Procesando Ligue 1 (Normalizado) con 2 clusters...
Procesando Serie A (Por 90 min) con 5 clusters...
Procesando Serie A (Normalizado) con 3 clusters...
Procesando Premier League (Por 90 min) con 3 clusters...
Procesando Premier League (Normalizado) con 3 clusters...
Procesando Global (Por 90 min) con 3 clusters...
Procesando Global (Normalizado) con 3 clusters...
✅ Exportación completada. Todos los CSVs listos para Power BI.


In [53]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import pandas as pd

resultados = []

liga_destino = "La Liga"

for liga_origen, config in ligas_config.items():
    if liga_origen == liga_destino:
        continue  # no tiene sentido comparar la liga consigo misma

    for tipo, valores in config.items():
        n_clusters = valores["clusters"]

        print(f"Prediciendo {liga_destino} usando {liga_origen} ({tipo}) con {n_clusters} clusters...")

        # Filtrar origen y destino
        if tipo == "Normalizado":
            X_origen = df_ready[df["liga"] == liga_origen]
            X_destino = df_ready[df["liga"] == liga_destino]
        else:  # Por 90 min
            X_origen = X_90[df["liga"] == liga_origen]
            X_destino = X_90[df["liga"] == liga_destino]

        # Comprobar que hay datos
        if X_origen.empty:
            print(f"⚠️ No hay datos para {liga_origen} ({tipo}), saltando...")
            continue

        # Etiquetas reales de origen
        y_origen = df.loc[X_origen.index, f"Cluster_{tipo.replace(' ', '')}_{liga_origen.replace(' ', '_')}"]

        # Entrenar en destino (La Liga)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        kmeans.fit(X_destino)

        # Predecir para equipos de origen en el modelo entrenado en La Liga
        y_pred = kmeans.predict(X_origen) + 1

        # Calcular ARI
        ari = adjusted_rand_score(y_origen, y_pred)

        # ---- Cálculo de separaciones usando groupby ----
        df_temp = pd.DataFrame({
            "cluster_origen": y_origen,
            "cluster_predicho": y_pred
        })

        separaciones = {}
        total_separados = 0

        # Para cada cluster de origen, identificar el cluster predicho más frecuente
        for cluster_real, grupo in df_temp.groupby("cluster_origen"):
            cluster_ref = grupo["cluster_predicho"].mode()[0]  # cluster más frecuente
            separados = (grupo["cluster_predicho"] != cluster_ref).sum()
            separaciones[int(cluster_real)] = int(separados)
            total_separados += separados

        resultados.append({
            "Liga_origen": liga_origen,
            "Tipo": tipo,
            "Clusters": n_clusters,
            "ARI_vs_LaLiga": ari,
            "Separaciones_por_cluster": separaciones,
            "Total_separados": int(total_separados),
            "Equipos_origen": len(X_origen)
        })

# Guardar resultados
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("predicciones_vs_laliga.csv", index=False)

print("\n✅ Predicciones completadas y guardadas en 'predicciones_vs_laliga.csv'.")


Prediciendo La Liga usando Femenino (Por 90 min) con 3 clusters...
Prediciendo La Liga usando Femenino (Normalizado) con 4 clusters...
Prediciendo La Liga usando Bundesliga (Por 90 min) con 4 clusters...
Prediciendo La Liga usando Bundesliga (Normalizado) con 3 clusters...
Prediciendo La Liga usando Ligue 1 (Por 90 min) con 3 clusters...
Prediciendo La Liga usando Ligue 1 (Normalizado) con 2 clusters...
Prediciendo La Liga usando Serie A (Por 90 min) con 5 clusters...
Prediciendo La Liga usando Serie A (Normalizado) con 3 clusters...
Prediciendo La Liga usando Premier League (Por 90 min) con 3 clusters...
Prediciendo La Liga usando Premier League (Normalizado) con 3 clusters...
Prediciendo La Liga usando Global (Por 90 min) con 3 clusters...
⚠️ No hay datos para Global (Por 90 min), saltando...
Prediciendo La Liga usando Global (Normalizado) con 3 clusters...
⚠️ No hay datos para Global (Normalizado), saltando...

✅ Predicciones completadas y guardadas en 'predicciones_vs_laliga.csv'.


In [47]:
import pandas as pd

# Cargar resultados
df_resultados = pd.read_csv("predicciones_vs_laliga.csv")

# Añadir columna de proporción de separados
df_resultados["Proporcion_separados"] = df_resultados["Total_separados"] / df_resultados["Equipos_origen"]

# Liga/tipo con mejor ARI
mejor_ari = df_resultados.loc[df_resultados["ARI_vs_LaLiga"].idxmax()]

# Liga/tipo con peor ARI
peor_ari = df_resultados.loc[df_resultados["ARI_vs_LaLiga"].idxmin()]

# Liga/tipo con mayor proporción de separados
mayor_sep = df_resultados.loc[df_resultados["Proporcion_separados"].idxmax()]

# Liga/tipo con menor proporción de separados
menor_sep = df_resultados.loc[df_resultados["Proporcion_separados"].idxmin()]

print("📊 Resumen de predicciones vs La Liga:\n")

print(f"Mejor ARI: {mejor_ari['Liga_origen']} ({mejor_ari['Tipo']}) -> ARI = {mejor_ari['ARI_vs_LaLiga']:.3f}, "
      f"Separados = {mejor_ari['Total_separados']}/{mejor_ari['Equipos_origen']}")

print(f"Peor ARI: {peor_ari['Liga_origen']} ({peor_ari['Tipo']}) -> ARI = {peor_ari['ARI_vs_LaLiga']:.3f}, "
      f"Separados = {peor_ari['Total_separados']}/{peor_ari['Equipos_origen']}")

print(f"Mayor proporción de separados: {mayor_sep['Liga_origen']} ({mayor_sep['Tipo']}) -> "
      f"{mayor_sep['Proporcion_separados']*100:.1f}% de equipos separados")

print(f"Menor proporción de separados: {menor_sep['Liga_origen']} ({menor_sep['Tipo']}) -> "
      f"{menor_sep['Proporcion_separados']*100:.1f}% de equipos separados")


📊 Resumen de predicciones vs La Liga:

Mejor ARI: Bundesliga (Normalizado) -> ARI = 0.947, Separados = 0/18
Peor ARI: Bundesliga (Por 90 min) -> ARI = 0.359, Separados = 5/18
Mayor proporción de separados: Bundesliga (Por 90 min) -> 27.8% de equipos separados
Menor proporción de separados: Bundesliga (Normalizado) -> 0.0% de equipos separados


## Añadimos variables PCA para poder graficar

In [57]:
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Diccionario para recorrer los dos DataFrames
dfs = {
    "Normalizado": df_ready_clusters,
    "Por90Min": X_90_clusters
}

for tipo, df_tipo in dfs.items():
    # Columnas numéricas para PCA (excluyendo ID y clusters)
    cols_excluir = ["Squad", "season", "liga"] + [c for c in df_tipo.columns if c.startswith("Cluster_")]
    X_pca_input = df_tipo.drop(columns=cols_excluir, errors="ignore")

    if X_pca_input.empty:
        print(f"⚠️ No hay columnas para PCA en {tipo}")
        continue

    # Imputar NaN con la media
    imputer = SimpleImputer(strategy="mean")
    X_clean = imputer.fit_transform(X_pca_input)

    # PCA 2D
    pca2 = PCA(n_components=2, random_state=42)
    X_pca2 = pca2.fit_transform(X_clean)
    df_tipo["PCA2D_1"] = X_pca2[:, 0]
    df_tipo["PCA2D_2"] = X_pca2[:, 1]

    # PCA 3D
    pca3 = PCA(n_components=3, random_state=42)
    X_pca3 = pca3.fit_transform(X_clean)
    df_tipo["PCA3D_1"] = X_pca3[:, 0]
    df_tipo["PCA3D_2"] = X_pca3[:, 1]
    df_tipo["PCA3D_3"] = X_pca3[:, 2]

    # PCA Radar (6D)
    pca6 = PCA(n_components=6, random_state=42)
    X_pca6 = pca6.fit_transform(X_clean)
    for i in range(6):
        df_tipo[f"PCARadar_{i+1}"] = X_pca6[:, i]

    print(f"✅ PCA aplicado y columnas añadidas para {tipo}")

# Exportar los DataFrames con clusters y PCA
df_ready_clusters.to_csv("df_ready_clusters_con_PCA.csv", index=False)
X_90_clusters.to_csv("X90_clusters_con_PCA.csv", index=False)


✅ PCA aplicado y columnas añadidas para Normalizado
✅ PCA aplicado y columnas añadidas para Por90Min


## Definir las variables más importantes
De esta forma sacamos las 6 variables más relevantes para poder sacar un buen gráfico de radar

In [58]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Guardaremos aquí las variables top por liga/tipo
top_vars_resultados = []

liga_destino = "La Liga"

for liga_origen, config in ligas_config.items():
    if liga_origen == liga_destino:
        continue  # no comparar contra sí misma

    for tipo, valores in config.items():
        n_clusters = valores["clusters"]

        print(f"\n🔎 Seleccionando variables más importantes para {liga_origen} ({tipo})...")

        # Filtrar origen usando df["liga"] como máscara
        if tipo == "Normalizado":
            mask = df["liga"] == liga_origen
            X_origen = df_ready.loc[mask].drop(columns=["liga"], errors="ignore")
        else:  # Por 90 min
            mask = df["liga"] == liga_origen
            X_origen = X_90.loc[mask].drop(columns=["liga"], errors="ignore")

        if X_origen.empty:
            print(f"⚠️ No hay datos para {liga_origen} ({tipo}), saltando...")
            continue

        # Etiquetas reales de origen
        cluster_col = f"Cluster_{tipo.replace(' ', '')}_{liga_origen.replace(' ', '_')}"
        y_origen = df.loc[X_origen.index, cluster_col]

        # Entrenar RandomForest para ver importancia de variables
        rf = RandomForestClassifier(random_state=42, n_estimators=300)
        rf.fit(X_origen, y_origen)

        importancias = pd.Series(rf.feature_importances_, index=X_origen.columns)
        top6 = importancias.sort_values(ascending=False).head(6)

        # Guardar resultados individuales
        for var, imp in top6.items():
            top_vars_resultados.append({
                "Liga_origen": liga_origen,
                "Tipo": tipo,
                "Variable": var,
                "Importancia": imp
            })

# ---- Guardar top6 por liga/tipo ----
df_top_vars = pd.DataFrame(top_vars_resultados)
df_top_vars.to_csv("top6_variables_por_liga.csv", index=False)

# ---- Calcular top6 globales ----
# Agrupamos todas las importancias de todas las ligas
df_global = df_top_vars.groupby("Variable")["Importancia"].mean().sort_values(ascending=False)

# Seleccionamos las 6 más importantes en general
top6_global = df_global.head(6).reset_index()
top6_global.columns = ["Variable", "Importancia_media"]

# Guardar en otro CSV
top6_global.to_csv("top6_variables_global.csv", index=False)

print("\n✅ Selección completada:")
print("- Guardado top6 por liga en 'top6_variables_por_liga.csv'")
print("- Guardado top6 global en 'top6_variables_global.csv'")
print("\n🏆 Top6 variables globales:\n", top6_global)



🔎 Seleccionando variables más importantes para Femenino (Por 90 min)...

🔎 Seleccionando variables más importantes para Femenino (Normalizado)...

🔎 Seleccionando variables más importantes para Bundesliga (Por 90 min)...

🔎 Seleccionando variables más importantes para Bundesliga (Normalizado)...

🔎 Seleccionando variables más importantes para Ligue 1 (Por 90 min)...

🔎 Seleccionando variables más importantes para Ligue 1 (Normalizado)...

🔎 Seleccionando variables más importantes para Serie A (Por 90 min)...

🔎 Seleccionando variables más importantes para Serie A (Normalizado)...

🔎 Seleccionando variables más importantes para Premier League (Por 90 min)...

🔎 Seleccionando variables más importantes para Premier League (Normalizado)...

🔎 Seleccionando variables más importantes para Global (Por 90 min)...
⚠️ No hay datos para Global (Por 90 min), saltando...

🔎 Seleccionando variables más importantes para Global (Normalizado)...
⚠️ No hay datos para Global (Normalizado), saltando...



In [62]:
# Columnas originales: solo numéricas, sin clusters ni PCA
cols_excluir = ["Squad", "season", "liga"] + [c for c in df_ready_clusters.columns if c.startswith("Cluster_") or c.startswith("PCA")]
X_clean_df = df_ready_clusters.drop(columns=cols_excluir, errors="ignore")

# Imputar NaN
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean")
X_clean = imputer.fit_transform(X_clean_df)

# PCA 6D
from sklearn.decomposition import PCA
pca6 = PCA(n_components=6, random_state=42)
pca6.fit(X_clean)

# Loadings: contribuciones de las variables originales
loadings = pd.DataFrame(pca6.components_.T, 
                        columns=[f"PC{i+1}" for i in range(6)], 
                        index=X_clean_df.columns)

# Ver las 5 variables que más aportan a cada componente
for i in range(6):
    print(f"\nComponente PC{i+1}:")
    print(loadings[f"PC{i+1}"].abs().sort_values(ascending=False).head(5))



Componente PC1:
Unnamed: 22_level_0 1/3_for    0.179687
SCA SCA_for                    0.168475
Team Success PPM_for           0.167466
GCA GCA_for                    0.167391
Unnamed: 3_level_0 Att_for     0.162141
Name: PC1, dtype: float64

Componente PC2:
Long Att_for             0.184150
Performance Recov_for    0.178647
Pass Types Sw_against    0.174213
Touches Def Pen_for      0.171977
Pass Types Dead_for      0.170235
Name: PC2, dtype: float64

Componente PC3:
Starts Mn/Start_for      0.245374
Tackles TklW_for         0.233870
Standard Dist_for        0.216071
Standard Dist_against    0.214579
Carries Dis_against      0.214130
Name: PC3, dtype: float64

Componente PC4:
Challenges Lost_against    0.238222
Touches Def 3rd_against    0.194594
Subs Subs_for              0.180375
Take-Ons Att_for           0.175425
Carries Mis_against        0.174881
Name: PC4, dtype: float64

Componente PC5:
Take-Ons Att_against               0.254500
Pass Types TB_against              0.237096
Cha

In [63]:
nombres_radar = {
    "PC1": "Creación Ofensiva / Eficacia",
    "PC2": "Transiciones / Recuperación",
    "PC3": "Defensa / Contribución Individual",
    "PC4": "Distribución / Posesión",
    "PC5": "Finalización / Remate",
    "PC6": "Defensa / Prevención de goles"
}
